In [114]:
import numpy as np 
import pandas as pd 
import gc
import matplotlib.pyplot as plt
import time
import seaborn as sns
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter('error', SettingWithCopyWarning)

gc.enable()

In [2]:
path='/home/baitong/pywork/RevenuePrediction/'

In [289]:
def readAllData(csvpath=path,n=18,name = 'train0'):
    fulldata = pd.DataFrame()
    for i in range(n):       
        print('loading data : ' + name + '%d'%i)
        df = pd.read_csv(csvpath+name +str(i) +'.csv',dtype={'fullVisitorId': 'str'})
        fulldata = pd.concat([fulldata,df],axis=0)
        del df
        gc.collect()
    return fulldata

In [374]:
%%time
train = readAllData()
test = readAllData(n=5,name = 'test0')

loading data : train00


/home/baitong/tensorflow/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (1,3,5,8,41,43,45,51,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


loading data : train01


/home/baitong/tensorflow/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (32,44,46,52,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
/home/baitong/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


loading data : train02


...

loading data : train016


/home/baitong/tensorflow/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (1,3,5,8,41,43,45,51,66) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


loading data : train017


/home/baitong/tensorflow/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (26,27,28,30,33,41,44,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


loading data : test00
loading data : test01
loading data : test02
loading data : test03
loading data : test04
CPU times: user 58.7 s, sys: 7.84 s, total: 1min 6s
Wall time: 1min 26s


In [375]:
useless = ['customDimensions_index','hits_appInfo.screenDepth','hits_dataSource','hits_eCommerceAction.option',
'hits_eCommerceAction.step','hits_exceptionInfo.isFatal','hits_index','hits_isEntrance','hits_isExit','hits_isInteraction',
'hits_item.currencyCode','hits_minute','hits_page.searchCategory','hits_page.searchKeyword',
'hits_promotionActionInfo.promoIsClick','hits_social.socialInteractionNetworkAction','hits_time',
           'hits_transaction.affiliation','hits_transaction.localTransactionRevenue','hits_transaction.localTransactionShipping',
'hits_transaction.transactionTax','hits_transaction.transactionRevenue','hits_type','hits_value',
'hits_transaction.transactionShipping','hits_transaction.transactionTax','trafficSource_adwordsClickInfo.adNetworkType',
'trafficSource_adwordsClickInfo.isVideoAd','trafficSource_adwordsClickInfo.page',
'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign','trafficSource_keyword']

In [376]:
train.drop(useless,inplace = True ,axis =1)
test.drop(useless,inplace = True ,axis =1)

In [377]:
train.columns

Index(['channelGrouping', 'customDimensions_value', 'date', 'device_browser',
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
       'fullVisitorId', 'geoNetwork_city', 'geoNetwork_continent',
       'geoNetwork_country', 'geoNetwork_metro', 'geoNetwork_networkDomain',
       'geoNetwork_region', 'geoNetwork_subContinent',
       'hits_appInfo.exitScreenName', 'hits_appInfo.landingScreenName',
       'hits_appInfo.screenName', 'hits_eCommerceAction.action_type',
       'hits_hitNumber', 'hits_hour', 'hits_item.transactionId',
       'hits_latencyTracking.domainLookupTime',
       'hits_latencyTracking.redirectionTime',
       'hits_latencyTracking.serverConnectionTime', 'hits_page.hostname',
       'hits_page.pagePath', 'hits_page.pagePathLevel1',
       'hits_page.pagePathLevel2', 'hits_page.pagePathLevel3',
       'hits_page.pagePathLevel4', 'hits_page.pageTitle',
       'hits_promotionActionInfo.promoIsView', 'hits_referer',
       'hits_social.hasSocial

In [378]:
num_feature = ['hits_hitNumber','hits_hour','totals_bounces','totals_hits', 'totals_newVisits', 'totals_pageviews',
       'totals_sessionQualityDim','totals_timeOnSite',
       'totals_transactionRevenue', 'totals_transactions','visitNumber']
exclude_feature = ['visitStartTime','date','fullVisitorId']

In [381]:
train = pd.read_csv(path+'train_useful.csv',dtype={'fullVisitorId': 'str'})
test = pd.read_csv(path+'test_useful.csv',dtype={'fullVisitorId': 'str'})

In [382]:
train['referralPath_depth']=train['trafficSource_referralPath'][train['trafficSource_referralPath'].notnull()].apply(lambda x:x.count('/'))
train['exitScreenName_depth']=train['hits_appInfo.exitScreenName'][train['hits_appInfo.exitScreenName'].notnull()].apply(lambda x:x.count('/'))
train['landingScreenName_depth']=train['hits_appInfo.landingScreenName'][train['hits_appInfo.landingScreenName'].notnull()].apply(lambda x:x.count('/'))
train['pagePath_depth']=train['hits_page.pagePath'][train['hits_page.pagePath'].notnull()].apply(lambda x:x.count('/'))
train['pagePathLevel1_depth']=train['hits_page.pagePathLevel1'][train['hits_page.pagePathLevel1'].notnull()].apply(lambda x:x.count('/'))
train['pagePathLevel2_depth']=train['hits_page.pagePathLevel2'][train['hits_page.pagePathLevel2'].notnull()].apply(lambda x:x.count('/'))
train['pagePathLevel3_depth']=train['hits_page.pagePathLevel3'][train['hits_page.pagePathLevel3'].notnull()].apply(lambda x:x.count('/'))
train['pagePathLevel4_depth']=train['hits_page.pagePathLevel4'][train['hits_page.pagePathLevel4'].notnull()].apply(lambda x:x.count('/'))
train['hits_referer_depth']=train['hits_referer'][train['hits_referer'].notnull()].apply(lambda x:x.count('/'))



In [383]:
test['referralPath_depth']=test['trafficSource_referralPath'][test['trafficSource_referralPath'].notnull()].apply(lambda x:x.count('/'))
test['exitScreenName_depth']=test['hits_appInfo.exitScreenName'][test['hits_appInfo.exitScreenName'].notnull()].apply(lambda x:x.count('/'))
test['landingScreenName_depth']=test['hits_appInfo.landingScreenName'][test['hits_appInfo.landingScreenName'].notnull()].apply(lambda x:x.count('/'))
test['pagePath_depth']=test['hits_page.pagePath'][test['hits_page.pagePath'].notnull()].apply(lambda x:x.count('/'))
test['pagePathLevel1_depth']=test['hits_page.pagePathLevel1'][test['hits_page.pagePathLevel1'].notnull()].apply(lambda x:x.count('/'))
test['pagePathLevel2_depth']=test['hits_page.pagePathLevel2'][test['hits_page.pagePathLevel2'].notnull()].apply(lambda x:x.count('/'))
test['pagePathLevel3_depth']=test['hits_page.pagePathLevel3'][test['hits_page.pagePathLevel3'].notnull()].apply(lambda x:x.count('/'))
test['pagePathLevel4_depth']=test['hits_page.pagePathLevel4'][test['hits_page.pagePathLevel4'].notnull()].apply(lambda x:x.count('/'))
test['hits_referer_depth']=test['hits_referer'][test['hits_referer'].notnull()].apply(lambda x:x.count('/'))
len(test.columns)

62

In [384]:
depth_feature = ['referralPath_depth','exitScreenName_depth','landingScreenName_depth',
                'pagePath_depth','pagePathLevel1_depth','pagePathLevel2_depth',
                'pagePathLevel3_depth','pagePathLevel4_depth','hits_referer_depth']
cat_feature = [f for f in train.columns if (f not in exclude_feature+num_feature+depth_feature)]
print(cat_feature)

['channelGrouping', 'customDimensions_value', 'device_browser', 'device_deviceCategory', 'device_isMobile', 'device_operatingSystem', 'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region', 'geoNetwork_subContinent', 'hits_appInfo.exitScreenName', 'hits_appInfo.landingScreenName', 'hits_appInfo.screenName', 'hits_eCommerceAction.action_type', 'hits_page.hostname', 'hits_page.pagePath', 'hits_page.pagePathLevel1', 'hits_page.pagePathLevel2', 'hits_page.pagePathLevel3', 'hits_page.pagePathLevel4', 'hits_page.pageTitle', 'hits_promotionActionInfo.promoIsView', 'hits_referer', 'hits_social.hasSocialSourceReferral', 'hits_social.socialNetwork', 'hits_transaction.currencyCode', 'hits_transaction.localTransactionTax', 'trafficSource_adContent', 'trafficSource_isTrueDirect', 'trafficSource_medium', 'trafficSource_referralPath', 'trafficSource_source']


In [385]:
##将缺失值表示统一
def replaceCatNa(df):
    Nalist = ['(not set)','0','not available in demo dataset','unknown.unknown','(not provided)','(none)']
    for col in cat_feature:
        df[col] = df[col].astype('str') 
        df[col] = df[col].apply(lambda x : np.nan if x in Nalist else x)
    return df

In [386]:
train = replaceCatNa(train)
test = replaceCatNa(test)

In [387]:
###将时间戳转换为标准时间表示
train.visitStartTime = pd.to_datetime(train.visitStartTime, unit='s')
test.visitStartTime = pd.to_datetime(test.visitStartTime, unit='s')
train.head()

,channelGrouping,customDimensions_value,date,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,fullVisitorId,geoNetwork_city,geoNetwork_continent,...,visitStartTime,referralPath_depth,exitScreenName_depth,landingScreenName_depth,pagePath_depth,pagePathLevel1_depth,pagePathLevel2_depth,pagePathLevel3_depth,pagePathLevel4_depth,hits_referer_depth
0,Organic Search,EMEA,20171016.0,Firefox,desktop,False,Windows,3162355547410993243,NaN,Europe,...,2017-10-17 00:00:50,NaN,3.0,3.0,3.0,2.0,2.0,1.0,NaN,3.0
1,Referral,North America,20171016.0,Chrome,desktop,False,Chrome OS,8934116514970143966,Cupertino,Americas,...,2017-10-16 17:51:47,6.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,8.0
2,Direct,North America,20171016.0,Chrome,mobile,True,Android,7992466427990357681,NaN,Americas,...,2017-10-17 00:53:33,NaN,3.0,1.0,1.0,1.0,NaN,NaN,NaN,3.0
3,Organic Search,EMEA,20171016.0,Chrome,desktop,False,Windows,9075655783635761930,NaN,Asia,...,2017-10-16 16:04:11,NaN,3.0,1.0,1.0,1.0,NaN,NaN,NaN,3.0
4,Organic Search,Central America,20171016.0,Chrome,desktop,False,Windows,6960673291025684308,NaN,Americas,...,2017-10-16 21:49:12,NaN,4.0,1.0,1.0,1.0,NaN,NaN,NaN,3.0


In [388]:
for df in [train, test]:
    df['weekday'] = df['visitStartTime'].dt.dayofweek.astype(object)
    df['month'] = df['visitStartTime'].dt.month   # it must not be included in features during learning!
#     df['day'] = df['visitStartTime'].dt.date       # it must not be included in features during learning!
    df['hours'] = df['visitStartTime'].dt.hour

In [389]:
###增加特征 ：是否为双休日
train['isWeekend'] = train['weekday'].apply(lambda x:1 if x>=5 else 0)
test['isWeekend'] = test['weekday'].apply(lambda x:1 if x>=5 else 0)

# train.to_csv(path+'train_useful.csv',index = False)
# test.to_csv(path+'test_useful.csv',index = False)

In [390]:
def addFeature(df):
    df['isWeekend'] = df['weekday'].apply(lambda x:1 if x>=5 else 0)
    predmis = df[df['totals_pageviews'].isnull()]['totals_hits'].values
    df.loc[(df['totals_pageviews'].isnull()),'totals_pageviews' ] = predmis
     ####增加特征：浏览器是否为Chrome
    df['isChrome'] = df['device_browser'].apply(lambda x:1 if x=='Chrome' else 0)   
    df['s_ismall'] = df['trafficSource_source'].apply(lambda x:1 if x=='mall.googleplex.com' else 0)
    df['s_isyoutube'] = df['trafficSource_source'].apply(lambda x:1 if x=='youtube.com' else 0)
    df['d_isdesktop'] = df['device_deviceCategory'].apply(lambda x:1 if x=='desktop' else 0)
    df['d_Macintosh'] = df['device_operatingSystem'].apply(lambda x:1 if x=='Macintosh' else 0)
    df['d_Macintosh'] = df['device_operatingSystem'].apply(lambda x:1 if x=='Macintosh' else 0)
    df['d_Linux'] = df['device_operatingSystem'].apply(lambda x:1 if x=='Linux' else 0)
    df['d_iOS'] = df['device_operatingSystem'].apply(lambda x:1 if x=='iOS' else 0)
    df['d_Android'] = df['device_operatingSystem'].apply(lambda x:1 if x=='Android' else 0)
    df['d_Windows Phone'] = df['device_operatingSystem'].apply(lambda x:1 if x=='Windows Phone' else 0)
    df['channel_referral'] = df['channelGrouping'].apply(lambda x:1 if x=='Referral' else 0)
    df['channel_Social'] = df['channelGrouping'].apply(lambda x:1 if x=='Social' else 0)
    def dealbrowser(s):
        if s == "Safari "or s == "Firefox":
            return "mainstream"
        elif s == "Chrome":
            return "Chrome"
        else:
            return "others"
    df['browser'] = df['device_browser'].apply(dealbrowser)
    df['m_referral'] = df['trafficSource_medium'].apply(lambda x:1 if x=='referral' else 0)  
    df['single_visit'] = df['visitNumber'].apply(lambda x:0 if x!=1 else 1)
    df['totals_hits'] = df['totals_hits'].astype('float') 
    df['totals_pageviews'] = df['totals_pageviews'].astype('float') 
    df['hits_ratio'] = df['totals_hits']/df['totals_pageviews']
    
    return df

In [391]:
train = addFeature(train)
test = addFeature(test)

In [392]:
train.columns

Index(['channelGrouping', 'customDimensions_value', 'date', 'device_browser',
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
       'fullVisitorId', 'geoNetwork_city', 'geoNetwork_continent',
       'geoNetwork_country', 'geoNetwork_metro', 'geoNetwork_networkDomain',
       'geoNetwork_region', 'geoNetwork_subContinent',
       'hits_appInfo.exitScreenName', 'hits_appInfo.landingScreenName',
       'hits_appInfo.screenName', 'hits_eCommerceAction.action_type',
       'hits_hitNumber', 'hits_hour', 'hits_page.hostname',
       'hits_page.pagePath', 'hits_page.pagePathLevel1',
       'hits_page.pagePathLevel2', 'hits_page.pagePathLevel3',
       'hits_page.pagePathLevel4', 'hits_page.pageTitle',
       'hits_promotionActionInfo.promoIsView', 'hits_referer',
       'hits_social.hasSocialSourceReferral', 'hits_social.socialNetwork',
       'hits_transaction.currencyCode', 'hits_transaction.localTransactionTax',
       'totals_bounces', 'totals_hits', 'totals_

In [393]:
num_feature = ['hits_hitNumber','hits_hour','totals_bounces','totals_hits', 'totals_newVisits', 'totals_pageviews',
       'totals_sessionQualityDim','totals_timeOnSite','hits_ratio','referralPath_depth', 'exitScreenName_depth', 'landingScreenName_depth',
       'pagePath_depth', 'pagePathLevel1_depth', 'pagePathLevel2_depth',
       'pagePathLevel3_depth', 'pagePathLevel4_depth', 'hits_referer_depth',
       'totals_transactionRevenue', 'totals_transactions','visitNumber']
exclude_feature = ['visitStartTime','date','fullVisitorId']
cat_feature = [f for f in train.columns if (f not in exclude_feature+num_feature)]
def dealMissingData(df):
    for c in df.columns:
        num_missing = df[c].isnull().sum() 
        if num_missing != 0:
            print("missing col is  ",c,'   missing num is',num_missing)
    print('*'*20)
    for col in num_feature:
        df[col].fillna(0,inplace=True)
    for col in cat_feature:
        df[col].fillna(0,inplace=True)
    for c in df.columns:
        num_missing = df[c].isnull().sum() 
        if num_missing != 0:
            print("now missing col is  ",c,'   missing num is',num_missing)
    print('---------------done---------------')
    return df

In [394]:
train = dealMissingData(train)
test = dealMissingData(test)

missing col is   device_browser    missing num is 19
missing col is   device_operatingSystem    missing num is 11815
missing col is   geoNetwork_city    missing num is 998826
missing col is   geoNetwork_continent    missing num is 2517
missing col is   geoNetwork_country    missing num is 2517
missing col is   geoNetwork_metro    missing num is 1319855
missing col is   geoNetwork_networkDomain    missing num is 768845
missing col is   geoNetwork_region    missing num is 982733
missing col is   geoNetwork_subContinent    missing num is 2517
missing col is   hits_hitNumber    missing num is 1215
missing col is   hits_hour    missing num is 1215
missing col is   hits_social.socialNetwork    missing num is 1352079
missing col is   totals_bounces    missing num is 836759
missing col is   totals_newVisits    missing num is 400907
missing col is   totals_sessionQualityDim    missing num is 835274
missing col is   totals_timeOnSite    missing num is 874294
missing col is   totals_transactionRe

In [395]:
def clearRare(columnname, limit = 1000):
    # you may search for rare categories in train, train&test, or just test
    #vc = pd.concat([train[columnname], test[columnname]], sort=False).value_counts()
    vc = test[columnname].value_counts()
    
    common = vc > limit
    common = set(common.index[common].values)
    print("Set", sum(vc <= limit), columnname, "categories to 'other';", end=" ")
    
    train.loc[train[columnname].map(lambda x: x not in common), columnname] = 'other'
    test.loc[test[columnname].map(lambda x: x not in common), columnname] = 'other'
    print("now there are", train[columnname].nunique(), "categories in train")

In [396]:
for col in cat_feature:
    clearRare(col)

Set 2 channelGrouping categories to 'other'; now there are 8 categories in train
Set 0 customDimensions_value categories to 'other'; now there are 6 categories in train
Set 53 device_browser categories to 'other'; now there are 11 categories in train
Set 1 device_deviceCategory categories to 'other'; now there are 3 categories in train
Set 1 device_isMobile categories to 'other'; now there are 2 categories in train
Set 16 device_operatingSystem categories to 'other'; now there are 8 categories in train
Set 462 geoNetwork_city categories to 'other'; now there are 42 categories in train
Set 2 geoNetwork_continent categories to 'other'; now there are 6 categories in train
Set 163 geoNetwork_country categories to 'other'; now there are 47 categories in train
Set 68 geoNetwork_metro categories to 'other'; now there are 15 categories in train
Set 15911 geoNetwork_networkDomain categories to 'other'; now there are 24 categories in train
Set 236 geoNetwork_region categories to 'other'; now the

In [397]:
train.set_index("visitStartTime", inplace=True)
test.set_index("visitStartTime", inplace=True)
train.sort_index(inplace=True)
test.sort_index(inplace=True)

In [398]:
train['time'] = train.index
test['time'] = test.index


In [399]:
###增加上一次会话和下一次会话的时间间隔，时间间隔越短说明用户购买商品欲望更强烈
df = pd.concat([train, test])
df.sort_values(['fullVisitorId', 'time'], ascending=True, inplace=True)
df['prev_session'] = (df['time'] - df[['fullVisitorId', 'time']].groupby('fullVisitorId')['time'].shift(1)).astype(np.int64) // 1e9 // 60 // 60
df['next_session'] = (df['time'] - df[['fullVisitorId', 'time']].groupby('fullVisitorId')['time'].shift(-1)).astype(np.int64) // 1e9 // 60 // 60
df.sort_index(inplace=True)

train = df[:len(train)]
test = df[len(train):]

/home/baitong/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [400]:
for df in [train, test]:
    for col in cat_feature:
        df[col] = df[col].astype('str')

In [401]:
##组合特征
for df in [train, test]:
    df['source_country'] = df['trafficSource_source'] + '_' + df['geoNetwork_country']
    df['browser_category'] = df['device_browser'] + '_' + df['device_deviceCategory']
    df['browser_os'] = df['device_browser'] + '_' + df['device_operatingSystem']
##组合特征
for df in [train, test]:
    df['device_deviceCategory_channelGrouping'] = df['device_deviceCategory'] + "_" + df['channelGrouping']
    df['channelGrouping_browser'] = df['device_browser'] + "_" + df['channelGrouping']
    df['channelGrouping_OS'] = df['device_operatingSystem'] + "_" + df['channelGrouping']
    
    for i in ['geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country','geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region','geoNetwork_subContinent']:
        for j in ['device_browser','device_deviceCategory', 'device_operatingSystem', 'trafficSource_source']:
            df[i + "_" + j] = df[i] + "_" + df[j]
    
    df['content_source'] = df['trafficSource_adContent'].astype(str) + "_" + df['source_country']
    df['medium_source'] = df['trafficSource_medium'] + "_" + df['source_country']


In [402]:
len(train.columns)

119

In [403]:
##针对用户增加点击浏览量均值特征，以及访问人数最大值特征
for feature in ["totals_hits","totals_pageviews",'totals_sessionQualityDim','totals_timeOnSite','hits_ratio',
               'referralPath_depth','exitScreenName_depth', 'landingScreenName_depth',
                'pagePath_depth', 'pagePathLevel1_depth', 'pagePathLevel2_depth', 
                'pagePathLevel3_depth', 'pagePathLevel4_depth','hits_referer_depth', 
                'totals_transactionRevenue', 'totals_transactions']:
    info = pd.concat([train, test], sort=False).groupby("fullVisitorId")[feature].mean()
    train["usermean_" + feature] = train.fullVisitorId.map(info)
    test["usermean_" + feature] = test.fullVisitorId.map(info)
    
for feature in ["visitNumber",'hits_hitNumber']:
    info = pd.concat([train, test], sort=False).groupby("fullVisitorId")[feature].max()
    train["usermax_" + feature] = train.fullVisitorId.map(info)
    test["usermax_" + feature] = test.fullVisitorId.map(info)

In [408]:
excluded = ['date','month','day','time','visitId','hits_item.transactionId','hits_transaction.transactionId'
           ,"fullVisitorId"]
# unique_f = ['hours', 'isChrome', 'isWeekend', 'm_referral', 's_ismall', 's_isyoutube', 'single_visit', 'totals_bounces']
cat_cols = [f for f in train.columns if (train[f].dtype == 'object' and f not in excluded)]
real_cols = [f for f in train.columns if (not f in cat_cols and f not in excluded)]
print("cat_cols num: ",len(cat_cols))
print("cat_cols: ",cat_cols)
print("real_cols num: ",len(real_cols))
print("real_cols: ",real_cols)

cat_cols num:  89
cat_cols:  ['browser', 'channelGrouping', 'channel_Social', 'channel_referral', 'customDimensions_value', 'd_Android', 'd_Linux', 'd_Macintosh', 'd_Windows Phone', 'd_iOS', 'd_isdesktop', 'device_browser', 'device_deviceCategory', 'device_isMobile', 'device_operatingSystem', 'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region', 'geoNetwork_subContinent', 'hits_appInfo.exitScreenName', 'hits_appInfo.landingScreenName', 'hits_appInfo.screenName', 'hits_eCommerceAction.action_type', 'hits_page.hostname', 'hits_page.pagePath', 'hits_page.pagePathLevel1', 'hits_page.pagePathLevel2', 'hits_page.pagePathLevel3', 'hits_page.pagePathLevel4', 'hits_page.pageTitle', 'hits_promotionActionInfo.promoIsView', 'hits_referer', 'hits_social.hasSocialSourceReferral', 'hits_social.socialNetwork', 'hits_transaction.currencyCode', 'hits_transaction.localTransactionTax', 'hours', 'isChrome', 'isWeekend', 'm_refe

In [409]:
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    test[col] = lbl.transform(list(test[col].values.astype('str')))

In [410]:
train.to_csv(path+'train_encoded.csv',index = False)
test.to_csv(path+'test_encoded.csv',index = False)

In [ ]:
train.to_csv(path+'train_encoded2.csv',index = True)
test.to_csv(path+'test_encode2.csv',index = True)

In [412]:
train['fullVisitorId'].head()

visitStartTime
2016-08-01 07:00:12    0423043652415339154
2016-08-01 07:04:26    8294721032567046680
2016-08-01 07:04:41    7718623669497357235
2016-08-01 07:06:01    4798058133221713505
2016-08-01 07:06:10    5076747981380011349
Name: fullVisitorId, dtype: object

In [413]:
users_period = [
    (20160801, 20170115),
    (20170115, 20170630),
    (20170701, 20171215),
    (20171216, 20180601),
    # Using the same months!
    (20170501, 20171015),
]

predict_period = [
    (20170301, 20170430),
    (20170715, 20170915),
    (20180201, 20180331),
    (20180715, 20180915),
    # Using the same months!
    (20171201, 20180131),
]

In [414]:
train = train[train['date']!='No']
train['date'] = train['date'].astype("float")

In [415]:
revenues_list = []

for i in range(5):
    print("\nPeriod", i+1)
    a, b = users_period[i]
    batch = train[(train.date >= a) & (train.date <= b)]
    batch_visitors = batch.fullVisitorId.unique()
    print("There are", len(batch_visitors), "visitors in 5.5 months")
    
    c, d = predict_period[i]
    pred = train[(train.date >= c) & (train.date <= d)]
    pred_visitors = pred.fullVisitorId.unique()
    print("There are", len(pred_visitors), "visitors in 2 months")
    # Returning visitors
    same_visitors = np.intersect1d(batch_visitors, pred_visitors)
    print("{} visitors returned or {:.2f}%".format(len(same_visitors), 100*len(same_visitors)/len(batch_visitors)))
    # Returning visitors revenue
    with_rev = pred[(pred.fullVisitorId.isin(same_visitors)) & (pred.totals_transactionRevenue > 0)]
    print("And only {} returning visitors have revenue or {:.2f}% from total".format(len(with_rev), 100*len(with_rev)/len(batch_visitors)))
    print("The total revenue for this users is U$$ {:.2f}".format(with_rev.totals_transactionRevenue.sum()/1000000))
    revenues_list.append(with_rev.copy(deep=True))


Period 1
There are 377186 visitors in 5.5 months
There are 111299 visitors in 2 months
1871 visitors returned or 0.50%
And only 210 returning visitors have revenue or 0.06% from total
The total revenue for this users is U$$ 102504.27

Period 2
There are 286797 visitors in 5.5 months
There are 129144 visitors in 2 months
3285 visitors returned or 1.15%
And only 357 returning visitors have revenue or 0.12% from total
The total revenue for this users is U$$ 88637.14

Period 3
There are 384946 visitors in 5.5 months
There are 140808 visitors in 2 months
2943 visitors returned or 0.76%
And only 148 returning visitors have revenue or 0.04% from total
The total revenue for this users is U$$ 15886.58

Period 4
There are 297074 visitors in 5.5 months
There are 0 visitors in 2 months
0 visitors returned or 0.00%
And only 0 returning visitors have revenue or 0.00% from total
The total revenue for this users is U$$ 0.00

Period 5
There are 329636 visitors in 5.5 months
There are 144049 visitors i

In [416]:
def splitByDate(df,users_period,predict_period):
    print('split by date...')
    print('users_period is ',users_period[0],'======>',users_period[1])
    batch = train[(df.date >= users_period[0]) & (df.date <= users_period[1])]
    print('predict_period is ',predict_period[0],'======>',predict_period[1])
    pred = train[(df.date >= predict_period[0]) & (df.date <= predict_period[1])]
    print("-----------------done--------------------")
    return batch,pred

In [417]:
train['fullVisitorId'].astype("str")
batch1,pred1 = splitByDate(train,users_period[0],predict_period[0])
batch2,pred2 = splitByDate(train,users_period[1],predict_period[1])
batch3,pred3 = splitByDate(train,users_period[2],predict_period[2])
batch4,pred4 = splitByDate(train,users_period[4],predict_period[4])

split by date...
users_period is  20160801 ======> 20170115
predict_period is  20170301 ======> 20170430
-----------------done--------------------
split by date...
users_period is  20170115 ======> 20170630
predict_period is  20170715 ======> 20170915
-----------------done--------------------
split by date...
users_period is  20170701 ======> 20171215
predict_period is  20180201 ======> 20180331
-----------------done--------------------
split by date...
users_period is  20170501 ======> 20171015
predict_period is  20171201 ======> 20180131
-----------------done--------------------


In [418]:
def extractTarget(df,user_period,pred_period,name):
#     print(user_period)
    batch,pred = splitByDate(df,user_period,pred_period)
    batch,pred = batch.copy(),pred.copy()
    print("提取交集...")
    ####提取交集 ： 即查看预测日期区间与训练集日期区间的用户交集，这个交集是二次光顾商店的人群
    pred["isIdhavetest"] = pred['fullVisitorId'].isin(batch['fullVisitorId'])
#     print(pred["isIdhavetest"].value_counts())
    pred = pred[pred["isIdhavetest"]==True]
    batch["isIdhavetest"]= batch['fullVisitorId'].isin(pred['fullVisitorId'])
#     print(batch["isIdhavetest"].value_counts())
     ####处理标签
    print("处理标签...")
    pred["isIdhavetest"] = pred["isIdhavetest"].apply(lambda x: 1 if x==True else 0)
    batch["isIdhavetest"] = batch["isIdhavetest"].apply(lambda x: 1 if x==True else 0)
    pred['target'] =pred['totals_transactionRevenue']
    pred = pred[['target','fullVisitorId',"isIdhavetest",'totals_transactionRevenue']]
    batch['target'] = 0
    batch = batch[['target','fullVisitorId',"isIdhavetest"]]
    ####按用户ID排序
    print("按用户ID排序并赋值...")
    pred = pred.groupby('fullVisitorId').sum()
    batch = batch.groupby('fullVisitorId').sum()
    tmp1 =pd.DataFrame()
    tmp2 = pd.DataFrame()
    tmp1['target'] = batch['target'][batch["isIdhavetest"]>=1] 
    tmp2['target'] = batch['target'][batch["isIdhavetest"]==0] 
    tmp1['target'] = pred['totals_transactionRevenue']#获取回头客的消费记录
    tmp2['target'] = 0#非回头客默认置零
    tmp = pd.concat([tmp1, tmp2])
    tmp.sort_index(inplace=True)
    print("保存...文件名: "+name+'.csv')
    batch['target'] = tmp['target']
    batch[['target']].to_csv('/home/baitong/pywork/RevenuePrediction/'+name+'.csv', index=True)
#     print(batch['target'].value_counts())
    print('------------------结束--------------------')

In [419]:
extractTarget(train,users_period[0],predict_period[0],'target1')
extractTarget(train,users_period[1],predict_period[1],'target2')
extractTarget(train,users_period[2],predict_period[2],'target3')
extractTarget(train,users_period[4],predict_period[4],'target4')

split by date...
users_period is  20160801 ======> 20170115
predict_period is  20170301 ======> 20170430
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target1.csv
------------------结束--------------------
split by date...
users_period is  20170115 ======> 20170630
predict_period is  20170715 ======> 20170915
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target2.csv
------------------结束--------------------
split by date...
users_period is  20170701 ======> 20171215
predict_period is  20180201 ======> 20180331
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target3.csv
------------------结束--------------------
split by date...
users_period is  20170501 ======> 20171015
predict_period is  20171201 ======> 20180131
-----------------done--------------------
提取交集...
处理标签...
按用户ID排序并赋值...
保存...文件名: target4.csv
------------------结束--------------------


In [421]:
# batch1.head()

In [213]:
batch1,pred1 = batch1.copy(),pred1.copy()
batch2,pred2 = batch2.copy(),pred2.copy()
batch3,pred3 = batch3.copy(),pred3.copy()
batch4,pred4 = batch4.copy(),pred4.copy()

In [ ]:
train_df = pd.concat([batch1,batch2,batch3,batch4],axis = 0)
label_df = pd.concat([pred1,pred2,pred3,pred4],axis = 0)